In [12]:
import pandas as pd
import numpy as np

# Replace with the paths to your CSV files:
serieA_df = pd.read_csv("C:\\Users\\sidbh\\Documents\\College\\DSE\\Datasets\\SerieA2022-2023.csv")
ligue1_df = pd.read_csv("C:\\Users\\sidbh\\Documents\\College\\DSE\\Datasets\\Ligue12022-2023.csv")
premier_league_df = pd.read_csv("C:\\Users\\sidbh\\Documents\\College\\DSE\\Datasets\\EPL2022-2023.csv")
laliga_df = pd.read_csv("C:\\Users\\sidbh\\Documents\\College\\DSE\\Datasets\\LaLiga2022-2023.csv")
bundesliga_df = pd.read_csv("C:\\Users\\sidbh\\Documents\\College\\DSE\\Datasets\\Bundesliga2022-2023.csv")

print(serieA_df.head())
print(ligue1_df.head())
print(premier_league_df.head())
print(laliga_df.head())
print(bundesliga_df.head())

         Date   Time    HomeTeam   AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
0  13/08/2022  17:30       Milan    Udinese     4     2   H     2     2   D   
1  13/08/2022  17:30   Sampdoria   Atalanta     0     2   A     0     1   A   
2  13/08/2022  19:45       Lecce      Inter     1     2   A     0     1   A   
3  13/08/2022  19:45       Monza     Torino     1     2   A     0     1   A   
4  14/08/2022  17:30  Fiorentina  Cremonese     3     2   H     2     1   H   

   ...  AvgC<2.5  AHCh  B365CAHH  B365CAHA  PCAHH  PCAHA  MaxCAHH  MaxCAHA  \
0  ...      2.16 -1.25      1.98      1.92   2.02   1.91     2.17     1.94   
1  ...      2.24  0.75      2.07      1.83   2.11   1.82     2.13     1.87   
2  ...      2.55  1.50      1.90      2.00   1.91   2.02     1.96     2.06   
3  ...      1.75  0.00      1.79      2.11   1.80   2.15     1.88     2.16   
4  ...      2.09 -1.00      2.02      1.88   2.06   1.87     2.07     1.97   

   AvgCAHH  AvgCAHA  
0     2.06     1.81  
1     2.06  

In [13]:
def remove_unwanted_columns(df):
    # Specify columns to keep
    columns_to_keep = ['HomeTeam', 'AwayTeam', 'FTR', 'FTHG', 'FTAG', 'HS', 'AS', 'HST', 'AST']

    # Drop all columns not in the list
    df = df[columns_to_keep]

    return df

# Apply to your DataFrames
serieA_df = remove_unwanted_columns(serieA_df.copy())  # Make a copy to avoid modifying the original
ligue1_df = remove_unwanted_columns(ligue1_df.copy())
premier_league_df = remove_unwanted_columns(premier_league_df.copy())
laliga_df = remove_unwanted_columns(laliga_df.copy())
bundesliga_df = remove_unwanted_columns(bundesliga_df.copy())

print(serieA_df.head())
print(ligue1_df.head())
print(premier_league_df.head())
print(laliga_df.head())
print(bundesliga_df.head())


     HomeTeam   AwayTeam FTR  FTHG  FTAG  HS  AS  HST  AST
0       Milan    Udinese   H     4     2  14   9    5    4
1   Sampdoria   Atalanta   A     0     2  11   9    1    3
2       Lecce      Inter   A     1     2  10  19    4    9
3       Monza     Torino   A     1     2  15  13    4    6
4  Fiorentina  Cremonese   H     3     2  27  13   11    4
    HomeTeam AwayTeam FTR  FTHG  FTAG  HS  AS  HST  AST
0       Nice    Lille   D     1     1   8  12    3    5
1  Marseille    Reims   H     2     1  14  11    4    4
2   Paris SG  Lorient   D     0     0  20   4    4    0
3      Brest     Lens   H     3     2  15  10    8    2
4   Clermont   Monaco   A     2     4  15  14    7    8
         HomeTeam       AwayTeam FTR  FTHG  FTAG  HS  AS  HST  AST
0  Crystal Palace        Arsenal   A     0     2  10  10    2    2
1          Fulham      Liverpool   D     2     2   9  11    3    4
2     Bournemouth    Aston Villa   H     2     0   7  15    3    2
3           Leeds         Wolves   H     2

In [19]:
import pandas as pd
import os

def calculate_points(row):
    if row['FTR'] == 'H':  # Home Win
        return 3, 0, 0
    elif row['FTR'] == 'A':  # Away Win
        return 0, 3, 0
    else:  # Draw
        return 1, 1, 0

# Directory to save the CSV files
save_dir = "C:\\Users\\sidbh\\Documents\\College\\DSE\\Datasets2"   

leagues_dataframes = [
    ('SerieA', serieA_df),
    ('Bundesliga', bundesliga_df),
    ('Premier League', premier_league_df),
    ('La Liga', laliga_df),
    ('Ligue 1', ligue1_df)
]

# Process each league's DataFrame
for league, df in leagues_dataframes:
    # Calculate points
    df[['HomePoints', 'AwayPoints', 'Losses']] = df.apply(calculate_points, axis=1, result_type='expand')

    # Merge unique home and away team lists
    team_list = pd.concat([df['HomeTeam'], df['AwayTeam']]).unique()

    # Create points table
    points_table = pd.DataFrame(index=team_list, 
                                columns=['Played', 'Wins', 'Draws', 'Losses', 'GF', 'GA', 'GD', 'Points'])
    points_table.fillna(0, inplace=True)

    for index, row in df.iterrows():
        home_team = row['HomeTeam']
        away_team = row['AwayTeam']
        home_goals = row['FTHG']
        away_goals = row['FTAG']
        result = row['FTR']

        points_table.loc[home_team, 'Played'] += 1
        points_table.loc[away_team, 'Played'] += 1
        points_table.loc[home_team, 'GF'] += home_goals
        points_table.loc[away_team, 'GF'] += away_goals
        points_table.loc[home_team, 'GA'] += away_goals
        points_table.loc[away_team, 'GA'] += home_goals

        if result == 'H':
            points_table.loc[home_team, 'Wins'] += 1
            points_table.loc[away_team, 'Losses'] += 1
        elif result == 'A':
            points_table.loc[away_team, 'Wins'] += 1
            points_table.loc[home_team, 'Losses'] += 1
        else:
            points_table.loc[home_team, 'Draws'] += 1
            points_table.loc[away_team, 'Draws'] += 1

    # Calculate Goal difference and total points
    points_table['GD'] = points_table['GF'] - points_table['GA']
    points_table['Points'] = (points_table['Wins'] * 3) + points_table['Draws']

    # Sort 
    points_table = points_table.sort_values(by=['Points', 'GD', 'GF'], ascending=False)

    # Save points table to CSV
    file_path = os.path.join(save_dir, f"{league}_points22_23.csv")
    points_table.to_csv(file_path)

    # Display points table
    print(f"\nLeague: {league}")
    print(points_table)


C:\Users\sidbh\AppData\Local\Temp\ipykernel_17984\2257256050.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  points_table.fillna(0, inplace=True)



League: SerieA
             Played  Wins  Draws  Losses  GF  GA  GD  Points
Napoli           38    28      6       4  77  28  49      90
Lazio            38    22      8       8  60  30  30      74
Inter            38    23      3      12  71  42  29      72
Juventus         38    22      6      10  56  33  23      72
Milan            38    20     10       8  64  43  21      70
Atalanta         38    19      7      12  66  48  18      64
Roma             38    18      9      11  50  38  12      63
Fiorentina       38    15     11      12  53  43  10      56
Bologna          38    14     12      12  53  49   4      54
Torino           38    14     11      13  42  41   1      53
Monza            38    14     10      14  48  52  -4      52
Udinese          38    11     13      14  47  48  -1      46
Sassuolo         38    12      9      17  47  61 -14      45
Empoli           38    10     13      15  37  49 -12      43
Salernitana      38     9     15      14  48  62 -14      42
Lecce   

C:\Users\sidbh\AppData\Local\Temp\ipykernel_17984\2257256050.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  points_table.fillna(0, inplace=True)



League: Bundesliga
               Played  Wins  Draws  Losses  GF  GA  GD  Points
Bayern Munich      34    21      8       5  92  38  54      71
Dortmund           34    22      5       7  83  44  39      71
RB Leipzig         34    20      6       8  64  41  23      66
Union Berlin       34    18      8       8  51  38  13      62
Freiburg           34    17      8       9  51  44   7      59
Leverkusen         34    14      8      12  57  49   8      50
Ein Frankfurt      34    13     11      10  58  52   6      50
Wolfsburg          34    13     10      11  57  48   9      49
Mainz              34    12     10      12  54  55  -1      46
M'gladbach         34    11     10      13  52  55  -3      43
FC Koln            34    10     12      12  49  54  -5      42
Hoffenheim         34    10      6      18  48  57  -9      36
Werder Bremen      34    10      6      18  51  64 -13      36
Bochum             34    10      5      19  40  72 -32      35
Augsburg           34     9      7 

C:\Users\sidbh\AppData\Local\Temp\ipykernel_17984\2257256050.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  points_table.fillna(0, inplace=True)



League: Premier League
                Played  Wins  Draws  Losses  GF  GA  GD  Points
Man City            38    28      5       5  94  33  61      89
Arsenal             38    26      6       6  88  43  45      84
Man United          38    23      6       9  58  43  15      75
Newcastle           38    19     14       5  68  33  35      71
Liverpool           38    19     10       9  75  47  28      67
Brighton            38    18      8      12  72  53  19      62
Aston Villa         38    18      7      13  51  46   5      61
Tottenham           38    18      6      14  70  63   7      60
Brentford           38    15     14       9  58  46  12      59
Fulham              38    15      7      16  55  53   2      52
Crystal Palace      38    11     12      15  40  49  -9      45
Chelsea             38    11     11      16  38  47  -9      44
Wolves              38    11      8      19  31  58 -27      41
West Ham            38    11      7      20  42  55 -13      40
Bournemouth     

C:\Users\sidbh\AppData\Local\Temp\ipykernel_17984\2257256050.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  points_table.fillna(0, inplace=True)



League: La Liga
             Played  Wins  Draws  Losses  GF  GA  GD  Points
Barcelona        38    28      4       6  70  20  50      88
Real Madrid      38    24      6       8  75  36  39      78
Ath Madrid       38    23      8       7  70  33  37      77
Sociedad         38    21      8       9  51  35  16      71
Villarreal       38    19      7      12  59  40  19      64
Betis            38    17      9      12  46  41   5      60
Osasuna          38    15      8      15  37  42  -5      53
Ath Bilbao       38    14      9      15  47  43   4      51
Mallorca         38    14      8      16  37  43  -6      50
Girona           38    13     10      15  58  55   3      49
Sevilla          38    13     10      15  47  54  -7      49
Vallecano        38    13     10      15  45  53  -8      49
Celta            38    11     10      17  43  53 -10      43
Valencia         38    11      9      18  42  45  -3      42
Getafe           38    10     12      16  34  45 -11      42
Cadiz  

C:\Users\sidbh\AppData\Local\Temp\ipykernel_17984\2257256050.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  points_table.fillna(0, inplace=True)



League: Ligue 1
             Played  Wins  Draws  Losses  GF  GA  GD  Points
Paris SG         29    19      9       1  69  25  44      66
Monaco           29    16      7       6  55  38  17      55
Brest            30    15      8       7  44  29  15      53
Lille            29    14     10       5  43  25  18      52
Nice             29    13      8       8  31  22   9      47
Lens             30    13      7      10  39  32   7      46
Rennes           30    11      9      10  44  36   8      42
Lyon             30    12      5      13  39  49 -10      41
Marseille        29    10     10       9  43  35   8      40
Reims            30    11      7      12  37  41  -4      40
Toulouse         30     9     10      11  36  39  -3      37
Montpellier      30     9     10      11  38  42  -4      37
Strasbourg       30     9      9      12  33  41  -8      36
Nantes           30     9      4      17  28  48 -20      31
Metz             30     8      5      17  31  49 -18      29
Le Havr

In [23]:
import pandas as pd
import os

# List of file paths
file_paths = [
    "C:\\Users\\sidbh\\Documents\\College\\DSE\\Datasets2\\Bundesliga_points22_23.csv",
    "C:\\Users\\sidbh\\Documents\\College\\DSE\\Datasets2\\La Liga_points22_23.csv",
    "C:\\Users\\sidbh\\Documents\\College\\DSE\\Datasets2\\Ligue 1_points22_23.csv",
    "C:\\Users\\sidbh\\Documents\\College\\DSE\\Datasets2\\Premier League_points22_23.csv",
    "C:\\Users\\sidbh\\Documents\\College\\DSE\\Datasets2\\SerieA_points22_23.csv"
]

# Initialize an empty dictionary to store team IDs
team_id_map = {}
id_counter = 0

# Loop through each file
for file_path in file_paths:
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Replace team names with unique IDs
    for team in df['Team'].unique():
        if team not in team_id_map:
            team_id_map[team] = id_counter
            id_counter += 1
    
    df['Team_ID'] = df['Team'].map(team_id_map)
    
    # Remove the 'Team' column
    df.drop(columns=['Team'], inplace=True)
    
    # Save the dataframe with unique team IDs and without the 'Team' column
    new_file_path = os.path.join(os.path.dirname(file_path), os.path.basename(file_path).replace('.csv', '_with_ids.csv'))
    df.to_csv(new_file_path, index=False)

    # Print the new dataframe
    print(df)



    Played  Wins  Draws  Losses  GF  GA  GD  Points  Team_ID
0       34    21      8       5  92  38  54      71        0
1       34    22      5       7  83  44  39      71        1
2       34    20      6       8  64  41  23      66        2
3       34    18      8       8  51  38  13      62        3
4       34    17      8       9  51  44   7      59        4
5       34    14      8      12  57  49   8      50        5
6       34    13     11      10  58  52   6      50        6
7       34    13     10      11  57  48   9      49        7
8       34    12     10      12  54  55  -1      46        8
9       34    11     10      13  52  55  -3      43        9
10      34    10     12      12  49  54  -5      42       10
11      34    10      6      18  48  57  -9      36       11
12      34    10      6      18  51  64 -13      36       12
13      34    10      5      19  40  72 -32      35       13
14      34     9      7      18  42  63 -21      34       14
15      34     7     12 

In [24]:
# Create a DataFrame to map team names to Team_ID numbers
team_id_mapping = pd.DataFrame(list(team_id_map.items()), columns=['Team', 'Team_ID'])

# Save the mapping DataFrame as a CSV file
mapping_file_path = os.path.join(os.path.dirname(file_paths[0]), 'Team_ID_Mapping.csv')
team_id_mapping.to_csv(mapping_file_path, index=False)

# Display the mapping DataFrame
print(team_id_mapping)


             Team  Team_ID
0   Bayern Munich        0
1        Dortmund        1
2      RB Leipzig        2
3    Union Berlin        3
4        Freiburg        4
..            ...      ...
91          Lecce       91
92         Verona       92
93         Spezia       93
94      Cremonese       94
95      Sampdoria       95

[96 rows x 2 columns]


In [27]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam

# List of file paths
file_paths = [
    "C:\\Users\\sidbh\\Documents\\College\\DSE\\Datasets2\\Bundesliga_points22_23_with_ids.csv",
    "C:\\Users\\sidbh\\Documents\\College\\DSE\\Datasets2\\La Liga_points22_23_with_ids.csv",
    "C:\\Users\\sidbh\\Documents\\College\\DSE\\Datasets2\\Ligue 1_points22_23_with_ids.csv",
    "C:\\Users\\sidbh\\Documents\\College\\DSE\\Datasets2\\Premier League_points22_23_with_ids.csv",
    "C:\\Users\\sidbh\\Documents\\College\\DSE\\Datasets2\\SerieA_points22_23_with_ids.csv"
]

# Concatenate data from all leagues into a single DataFrame
df_list = [pd.read_csv(file_path) for file_path in file_paths]
df = pd.concat(df_list)

# Define input and target columns
input_cols = ['Played', 'Wins', 'Draws', 'Losses', 'GF', 'GA', 'GD', 'Points']
target_col = 'Points'

# Normalize features using Min-Max scaling
scaler = MinMaxScaler()
df[input_cols] = scaler.fit_transform(df[input_cols])

# Convert DataFrame to supervised learning format for LSTM
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df_supervised = pd.DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in, 0, -1):
        cols.append(df_supervised.shift(i))
        names += [(f'{df_supervised.columns[j]}(t-{i})') for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df_supervised.shift(-i))
        if i == 0:
            names += [(f'{df_supervised.columns[j]}(t)') for j in range(n_vars)]
        else:
            names += [(f'{df_supervised.columns[j]}(t+{i})') for j in range(n_vars)]
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# Convert DataFrame to supervised learning format
df_supervised = series_to_supervised(df[input_cols])

# Split into input and output
X = df_supervised.drop(columns=[f'{target_col}(t)']).values
y = df_supervised[f'{target_col}(t)'].values

# Reshape input to be 3D [samples, timesteps, features]
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Fit the LSTM model
model.fit(X, y, epochs=100, batch_size=32, verbose=2)

# Placeholder DataFrame for next season data
df_next_season = pd.DataFrame({
    'Played': np.random.rand(20),  # Replace with actual data
    'Wins': np.random.rand(20),     # Replace with actual data
    'Draws': np.random.rand(20),    # Replace with actual data
    'Losses': np.random.rand(20),   # Replace with actual data
    'GF': np.random.rand(20),       # Replace with actual data
    'GA': np.random.rand(20),       # Replace with actual data
    'GD': np.random.rand(20),       # Replace with actual data
    'Points': np.zeros(20),         # Replace with actual data
    'Team_ID': np.arange(1, 21)     # Replace with actual data
})

# Normalize the next season data using the same scaler
df_next_season[input_cols] = scaler.transform(df_next_season[input_cols])

# Convert next season data to supervised learning format
df_next_season_supervised = series_to_supervised(df_next_season[input_cols])

# Extract input for next season
X_next_season = df_next_season_supervised.values.reshape((df_next_season_supervised.shape[0], 1, df_next_season_supervised.shape[1]))

# Make predictions for the next season
predicted_points = model.predict(X_next_season)

# Create a DataFrame with predicted points for each team in the next season
predicted_df = pd.DataFrame({'Team_ID': df_next_season['Team_ID'], 'Predicted_Points': predicted_points.flatten()})

# Print the predicted points table
print("Predicted Points for Next Season:")
print(predicted_df)


Epoch 1/100


c:\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 - 2s - 691ms/step - loss: 0.1007
Epoch 2/100
3/3 - 0s - 17ms/step - loss: 0.0817
Epoch 3/100
3/3 - 0s - 20ms/step - loss: 0.0643
Epoch 4/100
3/3 - 0s - 20ms/step - loss: 0.0501
Epoch 5/100
3/3 - 0s - 20ms/step - loss: 0.0389
Epoch 6/100
3/3 - 0s - 19ms/step - loss: 0.0319
Epoch 7/100
3/3 - 0s - 15ms/step - loss: 0.0273
Epoch 8/100
3/3 - 0s - 18ms/step - loss: 0.0258
Epoch 9/100
3/3 - 0s - 16ms/step - loss: 0.0253
Epoch 10/100
3/3 - 0s - 18ms/step - loss: 0.0239
Epoch 11/100
3/3 - 0s - 19ms/step - loss: 0.0214
Epoch 12/100
3/3 - 0s - 16ms/step - loss: 0.0186
Epoch 13/100
3/3 - 0s - 17ms/step - loss: 0.0161
Epoch 14/100
3/3 - 0s - 18ms/step - loss: 0.0140
Epoch 15/100
3/3 - 0s - 14ms/step - loss: 0.0126
Epoch 16/100
3/3 - 0s - 18ms/step - loss: 0.0113
Epoch 17/100
3/3 - 0s - 17ms/step - loss: 0.0100
Epoch 18/100
3/3 - 0s - 17ms/step - loss: 0.0089
Epoch 19/100
3/3 - 0s - 21ms/step - loss: 0.0078
Epoch 20/100
3/3 - 0s - 33ms/step - loss: 0.0072
Epoch 21/100
3/3 - 0s - 19ms/step - loss

ValueError: Exception encountered when calling LSTMCell.call().

[1mDimensions must be equal, but are 16 and 15 for '{{node sequential_5_1/lstm_2_1/lstm_cell_1/MatMul}} = MatMul[T=DT_FLOAT, grad_a=false, grad_b=false, transpose_a=false, transpose_b=false](sequential_5_1/lstm_2_1/strided_slice_1, sequential_5_1/lstm_2_1/lstm_cell_1/Cast/ReadVariableOp)' with input shapes: [19,16], [15,200].[0m

Arguments received by LSTMCell.call():
  • inputs=tf.Tensor(shape=(19, 16), dtype=float32)
  • states=('tf.Tensor(shape=(19, 50), dtype=float32)', 'tf.Tensor(shape=(19, 50), dtype=float32)')
  • training=False